In [58]:
import pandas
from pandas import DataFrame
from scipy.stats import pearsonr
import numpy as np
import utils
import os

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation
np.set_printoptions(precision=2)

In [44]:
dataPath = os.path.join(os.getenv("HOME"), 
                        'Project/WomenIndustrySurveyHolzblatt/Data/DataForClustering/slidersLikertByItem.csv')

data = pandas.read_csv(dataPath, sep=',')
# Make the question column the index, so that
# all operations on the dataframe only address
# the numeric scores:
data = data.set_index('question')
# Replace 0s with their column's median.
data = utils.replaceZerosDataFrame(data, direction='row')

In [56]:
def corrcoef_loop(matrix):
    rows, cols = matrix.shape[0], matrix.shape[1]
    r = np.ones(shape=(rows, rows))
    p = np.ones(shape=(rows, rows))
    for i in range(rows):
        for j in range(i+1, rows):
            r_, p_ = pearsonr(matrix[i], matrix[j])
            r[i, j] = r[j, i] = r_
            p[i, j] = p[j, i] = p_
    return r, p

In [75]:
corrMatrixPath = os.path.splitext(dataPath)[0] + 'Corrs.xlsx'
corrMatrixPath

'/Users/paepcke/Project/WomenIndustrySurveyHolzblatt/Data/DataForClustering/slidersLikertByItemCorrs.xlsx'

In [80]:
(r, p) = corrcoef_loop(data.values)
#data.index
crossCorrR = DataFrame(r, columns=data.index, index=data.index)
writer = pandas.ExcelWriter(corrMatrixPath)
crossCorrR.to_excel(writer, 'SlidersLikertCorrelationsR')
# Save the p values in a separate sheet:
crossCorrP = DataFrame(p, columns=data.index, index=data.index)
crossCorrP.to_excel(writer, 'SlidersLikertCorrelationsP')
writer.save()